# Playing with SPM.mat files in Python

In [1]:
import scipy.io as sio
import numpy as np
from itertools import groupby
from operator import itemgetter
from os.path import dirname

In [2]:
def get_consecutive(index):
    ranges = []
    for key, group in groupby(enumerate(index),
                              lambda (index, item): index - item):
        group = map(itemgetter(1), group)
        if len(group) > 1:
            ranges.append(xrange(group[0], group[-1]))
        else:
            ranges.append(group[0])
    return ranges[0]

In [3]:
def make_lss_vectors(vectors):
    all_names, all_onsets, all_durations = vectors
    lss_names = []
    lss_onsets = []
    lss_durations = []
    for i in range(len(all_names)):
        temp_all_names = list(all_names)
        temp_all_onsets = list(all_onsets)
        temp_all_durations = list(all_durations)
        name = temp_all_names.pop(i)
        onsets = temp_all_onsets.pop(i)
        durations = temp_all_durations.pop(i)
        for trial in range(len(onsets)):
            trial_name = '{0}_{1}'.format(name, trial)
            all_trial_names = [trial_name, name] + temp_all_names
            lss_names.append(all_trial_names)

            temp_onsets = list(onsets)
            onset = temp_onsets.pop(trial)
            all_trial_onsets = [[onset], temp_onsets] + temp_all_onsets
            lss_onsets.append(all_trial_onsets)

            temp_durations = list(durations)
            duration = temp_durations.pop(trial)
            all_trial_durations = [[duration], temp_durations] + temp_all_durations
            lss_durations.append(all_trial_durations)
    lss_vectors = [lss_names, lss_onsets, lss_durations]
    return lss_vectors

In [4]:
spm_file = '/home/tsalo006/misc-fmri-code/beta-example/first_level/SPM.mat'

In [5]:
LV = sio.loadmat(spm_file)

In [6]:
files = LV['SPM'][0]['xY'][0]['P'][0][0]  # <- one of these may point to run
sessions = LV['SPM'][0]['Sess'][0][0]
print('Modeling {0} timepoints across {1} sessions.'.format(len(files), len(sessions)))

Modeling 200 timepoints across 2 sessions.


In [7]:
# Find shared filepath across files
idx = range(len(files[0]))
f0 = np.array(list(files[0]), str)
for f in files:
    fl = np.array(list(f), str)
    min_len = np.min((len(f0), len(fl)))
    temp_idx = np.where(f0[:min_len]==fl[:min_len])
    idx = np.intersect1d(idx, temp_idx)

In [8]:
idx2 = get_consecutive(idx)
shared_str = ''.join([f0[i] for i in idx2])
old_path = dirname(shared_str)

temp_dir = '/scratch/tsalo006'
new_files = [f.replace(old_path, temp_dir) for f in files]
new_data_dirs = list(set([dirname(f) for f in new_files]))
# for data_dir in new_data_dirs:
#     if not isdir(data_dir):
#         mkdir(data_dir)
# 
# for i, f in enumerate(files):
#     copyfile(f, new_files[i])

for i_sess in range(len(sessions)):
    row_idx = sessions[i_sess]['row'][0] - 1  # don't forget python zero-indexes
    sess_files = [nf for j, nf in enumerate(new_files) if j in row_idx]
    covariates = sessions[i_sess]['C'][0]['C'][0]
    orig_names = []
    orig_onsets = []
    orig_durations = []
    for cond in sessions[i_sess]['U'][0]:
        orig_names.append(cond['name'][0][0][0])
        orig_onsets.append(np.squeeze(cond['ons']))
        orig_durations.append(np.squeeze(cond['dur']))
    params = [orig_names, orig_onsets, orig_durations]
    lss_vectors = make_lss_vectors(params)
    lss_names, lss_onsets, lss_durations = lss_vectors
    print lss_names

[['Impulses_0', u'Impulses'], ['Impulses_1', u'Impulses'], ['Impulses_2', u'Impulses'], ['Impulses_3', u'Impulses'], ['Impulses_4', u'Impulses'], ['Impulses_5', u'Impulses'], ['Impulses_6', u'Impulses'], ['Impulses_7', u'Impulses'], ['Impulses_8', u'Impulses'], ['Impulses_9', u'Impulses']]
[['Impulses_0', u'Impulses'], ['Impulses_1', u'Impulses'], ['Impulses_2', u'Impulses'], ['Impulses_3', u'Impulses'], ['Impulses_4', u'Impulses'], ['Impulses_5', u'Impulses'], ['Impulses_6', u'Impulses'], ['Impulses_7', u'Impulses'], ['Impulses_8', u'Impulses'], ['Impulses_9', u'Impulses']]
